# Data Prep - Census Data
This notebook is the second part to the data preparation phase for this project. I will perform a series of manipulations on sociodemographic data from the 2010's Brazilian Census, the latest census conducted in the country. This is a complex process with many stages and will involve both careful manipulations of the raw data as well as handling of geospatial data.

In order to perform these manipulations, we will use both `pyspark` and an extension called `Apache Sedona` (formerly known as `geospark`).

In [1]:
# installing the requirements:
!pip install -r ../configs/dependencies/dataprep_requirements.txt >> ../configs/dependencies/package_installation.txt

In [194]:
# loading the magic commands:
%load_ext lab_black
%load_ext autoreload
%autoreload 2

The lab_black extension is already loaded. To reload it, use:
  %reload_ext lab_black
The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [195]:
###### Loading the necessary libraries #########

# PySpark dependencies:s
import pyspark
from pyspark import SparkConf
from pyspark import SparkContext
from pyspark.sql import SparkSession
from pyspark.sql import SQLContext
import pyspark.sql.functions as F
from pyspark.sql.functions import udf
import pyspark.sql.types as T
from pyspark.sql.window import Window

# Sedona dependencies:
from sedona.utils.adapter import Adapter
from sedona.register import SedonaRegistrator
from sedona.utils import KryoSerializer, SedonaKryoRegistrator
from sedona.core.SpatialRDD import SpatialRDD
from sedona.core.formatMapper.shapefileParser import ShapefileReader
from sedona.core.formatMapper import GeoJsonReader

# database utilities:
from sqlalchemy import create_engine
import sqlite3 as db
import pandas as pd
from tqdm import tqdm
import geopandas as gpd
import fiona

# plotting and data visualization:
import matplotlib.pyplot as plt
import seaborn as sns
from IPython.display import HTML, Image

# other relevant libraries:
import warnings
import unidecode
import inflection
import unicodedata
from datetime import datetime, timedelta
from functools import partial
import json
import re
import os
from glob import glob
import shutil
import itertools
import chardet

# importing the atlas utilities:
from atlasutils import (
    save_to_filesystem,
    save_as_table,
    rotate_xticks,
    get_file_encoding,
    normalize_entities,
    normalize_column_name,
    apply_category_map,
    standardize_variable_names,
    get_null_columns,
    replace_decimal_separator,
    convert_to_geopandas,
    drop_invalid_census_columns,
    clean_census_column_name,
    get_file_crs,
    get_column_values,
)


# setting global parameters for visualizationsss:
warnings.filterwarnings("ignore")
pd.set_option("display.precision", 4)
pd.set_option("display.float_format", lambda x: "%.2f" % x)

# 0. Configuring Spark

In [199]:
# function to encapsulate standard spark configurations:
def init_spark(app_name):

    spark = (
        SparkSession.builder.appName(app_name)
        .config("spark.files.overwrite", "true")
        .config("spark.serializer", KryoSerializer.getName)
        .config("spark.kryo.registrator", SedonaKryoRegistrator.getName)
        .config(
            "spark.jars.packages",
            "org.apache.sedona:sedona-python-adapter-3.0_2.12:1.0.1-incubating,"
            "org.datasyslab:geotools-wrapper:geotools-24.1",
        )
        .config("spark.sql.repl.eagerEval.enabled", True)
        .config("spark.sql.repl.eagerEval.maxNumRows", 5)
        .config("spark.sql.legacy.timeParserPolicy", "LEGACY")
        .config("spark.sql.parquet.compression.codec", "gzip")
        .config("sedona.global.charset", "utf8")
        .config("sedona.global.index", "true")
        .enableHiveSupport()
        .getOrCreate()
    )

    SedonaRegistrator.registerAll(spark)

    return spark

In [200]:
# init the spark session:
spark = init_spark("SP Atlas - IBGE Census")

In [201]:
# verifying the session status:
spark

# 1. Loading and Inspecting the Data
We will first load the Polygon geospatial data that we will later use to match to the `(lat, long)` paths in the listings datasets.

## 1.1 Sector Polygons
The lowest level of data aggregation in the Census is a Sector. A sector is a specific area in which researchers collect census data. These sectors are contiguous areas and IBGE provides the Shapefiles for them. This will allow us to match the Listings data to the census sector where they can be located.

In [202]:
# loading the raw dataset:
RAW_DATA_DIR = "../data/raw/"

# shapefiles for sectors:
sector_rdd = ShapefileReader.readToGeometryRDD(
    sc=spark, inputPath=RAW_DATA_DIR + "sp_layers/census_sectors/*"
)

df_sector = Adapter.toDf(sector_rdd, spark)

# verifying the sectors:
df_sector

geometry,ID,CD_GEOCODI,TIPO,CD_GEOCODS,NM_SUBDIST,CD_GEOCODD,NM_DISTRIT,CD_GEOCODM,NM_MUNICIP,NM_MICRO,NM_MESO,CD_GEOCODB,NM_BAIRRO,ID1
POLYGON ((-46.410...,98237,354100005000009,URBANO,35410000500,,354100005,PRAIA GRANDE,3541000,PRAIA GRANDE,SANTOS,METROPOLITANA DE ...,354100005001,Boqueirão,1
POLYGON ((-46.416...,98232,354100005000004,URBANO,35410000500,,354100005,PRAIA GRANDE,3541000,PRAIA GRANDE,SANTOS,METROPOLITANA DE ...,354100005001,Boqueirão,2
POLYGON ((-46.412...,98230,354100005000002,URBANO,35410000500,,354100005,PRAIA GRANDE,3541000,PRAIA GRANDE,SANTOS,METROPOLITANA DE ...,354100005001,Boqueirão,3
POLYGON ((-46.411...,98229,354100005000001,URBANO,35410000500,,354100005,PRAIA GRANDE,3541000,PRAIA GRANDE,SANTOS,METROPOLITANA DE ...,354100005001,Boqueirão,4
POLYGON ((-46.413...,98231,354100005000003,URBANO,35410000500,,354100005,PRAIA GRANDE,3541000,PRAIA GRANDE,SANTOS,METROPOLITANA DE ...,354100005001,Boqueirão,5


In [203]:
# selecting the relevant columns:
df_sector = df_sector.select(
    F.col("ID").alias("id"),
    F.col("geometry"),
    F.col("CD_GEOCODI").alias("sector_code"),
    F.col("CD_GEOCODM").alias("city_code"),
    F.col("CD_GEOCODB").alias("neighborhood_code"),
    F.col("NM_MUNICIP").alias("city"),
    F.col("NM_BAIRRO").alias("neighborhood"),
    F.col("TIPO").alias("sector_type"),
)

# filtering just São Paulo:
df_sector = df_sector.filter(F.col("city") == "SÃO PAULO")

## 1.2 Area of Ponderation

In [204]:
# reading the area of ponderation files:
ap_rdd = ShapefileReader.readToGeometryRDD(
    sc=spark, inputPath=RAW_DATA_DIR + "sp_layers/census_ponderations/*"
)

df_ap = Adapter.toDf(ap_rdd, spark)

# adding the geometry dataframes to the SQL Context:
df_ap.createOrReplaceTempView("tb_ponderation")

# verifying the sectors:
df_ap

geometry,ID,AREA,COD_AED,COD_AED_S
POLYGON ((317854....,158,10.450821,3550308005243,243
POLYGON ((320682....,159,13.241315,3550308005242,242
POLYGON ((325497....,160,4.180473,3550308005309,309
POLYGON ((326306....,117,11.936847,3550308005189,189
POLYGON ((328654....,1,7.321331,3550308005127,127


In [205]:
Q_AP_CONVERSION = """
SELECT 
    ST_FlipCoordinates(ST_Transform(A.geometry, 'epsg:29193','epsg:4326')) as geometry,
    A.AREA as ponderation_area,
    A.COD_AED as ponderation_area_code
FROM tb_ponderation as A
"""

# converting the CRS to the standard format:
df_ap = spark.sql(Q_AP_CONVERSION)

# adding the geometry dataframes to the SQL Context:
df_ap.createOrReplaceTempView("tb_ponderation")

In [206]:
# verifying the results:
df_ap

geometry,ponderation_area,ponderation_area_code
POLYGON ((-46.782...,10.450821,3550308005243
POLYGON ((-46.755...,13.241315,3550308005242
POLYGON ((-46.708...,4.180473,3550308005309
POLYGON ((-46.700...,11.936847,3550308005189
POLYGON ((-46.677...,7.321331,3550308005127


## 1.3 Neighborhoods

In [207]:
# reading the neighborhoods files:
nb_rdd = ShapefileReader.readToGeometryRDD(
    sc=spark, inputPath=RAW_DATA_DIR + "sp_layers/neighborhoods/*"
)

df_nb = Adapter.toDf(nb_rdd, spark)
df_nb.createOrReplaceTempView("tb_neighborhood")

# verifying the sectors:
df_nb

geometry,Name,descriptio
MULTIPOLYGON (((-...,Alto da Riviera,ALTO DA RIVIERA
POLYGON ((-46.589...,Alto da Mooca,ALTO DA MOOCA
POLYGON ((-46.719...,Alto da Lapa,ALTO DA LAPA
POLYGON ((-46.629...,Vila Agua Funda,VILA AGUA FUNDA
POLYGON ((-46.622...,Agua Fria,AGUA FRIA


## 1.4 Districts

In [208]:
# reading the neighborhoods files:
district_rdd = ShapefileReader.readToGeometryRDD(
    sc=spark, inputPath=RAW_DATA_DIR + "sp_layers/districts/*"
)

df_district = Adapter.toDf(district_rdd, spark)

# verifying the sectors:
df_district

geometry,CLASSID,FEATID,REVISIONNU,NOME_DIST,SIGLA_DIST,COD_DIST,COD_SUB,DATA_CRIAC,USUARIO_ID
POLYGON ((352436....,4.00000,8583485.00000,1.00000,JOSE BONIFACIO,JBO,47,27,20070319,0.00000
POLYGON ((320696....,4.00000,8583484.00000,1.00000,JD SAO LUIS,JDS,46,18,20070319,0.00000
POLYGON ((349461....,4.00000,8583445.00000,1.00000,ARTUR ALVIM,AAL,5,21,20070319,0.00000
POLYGON ((320731....,4.00000,8583479.00000,1.00000,JAGUARA,JAG,40,08,20070319,0.00000
POLYGON ((338651....,4.00000,8583437.00000,1.00000,VILA PRUDENTE,VPR,93,29,20070319,0.00000


In [209]:
# it looks like the file's geometry is not the standard we want (ESPG:4326), let's find out what is it:
get_file_crs(RAW_DATA_DIR + "sp_layers/districts/")

<Projected CRS: EPSG:29193>
Name: SAD69 / UTM zone 23S
Axis Info [cartesian]:
- E[east]: Easting (metre)
- N[north]: Northing (metre)
Area of Use:
- name: Brazil - between 48°W and 42°W, northern and southern hemispheres, onshore and offshore.
- bounds: (-48.0, -33.5, -42.0, 5.13)
Coordinate Operation:
- name: UTM zone 23S
- method: Transverse Mercator
Datum: South American Datum 1969
- Ellipsoid: GRS 1967 Modified
- Prime Meridian: Greenwich

We will need to convert the dataset to a standard CRS (coordinate reference system), which, in our case, is `espg:4326`.

In [210]:
# dropping the unnecessary columns:
df_district = df_district.drop(
    "CLASSID", "FEATID", "REVISIONNU", "DATA_CRIAC", "USUARIO_ID"
)

df_district.createOrReplaceTempView("tb_district")

# converting the coordinate system in the district file:
Q_DISTRICT_CONVERSION = """
SELECT 
    ST_FlipCoordinates(ST_Transform(A.geometry, 'epsg:29193','epsg:4326')) as geometry,
    A.NOME_DIST as district_name,
    A.SIGLA_DIST as district_abbreviation,
    A.COD_DIST as district_code,
    A.COD_SUB as subdistrict_code
FROM tb_district as A
"""

df_district = spark.sql(Q_DISTRICT_CONVERSION)

## 1.5 Zip Codes

In [211]:
# shapefiles for sectors:
zipcode_rdd = ShapefileReader.readToGeometryRDD(
    sc=spark, inputPath=RAW_DATA_DIR + "sp_layers/logradouros/*"
)

df_zipcode = Adapter.toDf(zipcode_rdd, spark)
df_zipcode.createOrReplaceTempView("tb_zipcode")

# verifying the zipcode shapes::
df_zipcode

geometry,ID,LENGTH,CODL,SEG,ANGULO,ORIENTA_O,COD_ORI,LEFT_ZIP,RIGHT_ZIP,START_LEFT,END_LEFT,START_RIGH,END_RIGHT,CEP_D,CEP_E,TITULO,PREP,NAME,BAIRRO_D,BAIRRO_E
LINESTRING (31949...,1,227.218229000000010,731960,SG731960001,170,LESTE-OESTE,O,5547,5547,1,227,2,226,05547020,05547020,,,ERNESTO PAGLIA,JARDIM ROSA MARIA,JARDIM ROSA MARIA
LINESTRING (31949...,2,100.109236999999990,753718,SG753718003,294,NORTE-SUL,S,5549,5549,111,209,110,208,05549120,05549120,PRFA.,,MARIA OSORIO TEIX...,JARDIM DAS ESMERA...,JARDIM DAS ESMERA...
LINESTRING (31955...,3,76.883336000000000,734519,SG734519005,135,LESTE-OESTE,O,5549,5549,281,355,280,354,05549080,05549080,,,VITORIANO DA SILVA,JARDIM DAS ESMERA...,JARDIM DAS ESMERA...
LINESTRING (31954...,4,50.402368000000003,753718,SG753718002,208,LESTE-OESTE,O,5549,5549,61,109,60,108,05549120,05549120,PRFA.,,MARIA OSORIO TEIX...,JARDIM DAS ESMERA...,JARDIM DAS ESMERA...
LINESTRING (31959...,5,59.831297999999997,753718,SG753718001,205,LESTE-OESTE,O,5549,5549,1,59,2,58,05549120,05549120,PRFA.,,MARIA OSORIO TEIX...,JARDIM DAS ESMERA...,JARDIM DAS ESMERA...


In [212]:
# converting the coordinate system in the zipcode file:
Q_ZIPCODE_CONVERSION = """
WITH zip_left as (
  SELECT 
    DISTINCT A.CEP_E as zipcode,
    ST_FlipCoordinates(ST_Transform(A.geometry, 'epsg:29193','epsg:4326')) as geometry,
    A.NAME as street_name,
    A.LENGTH as street_length
    FROM tb_zipcode as A  
),

zip_right as (
    SELECT
        DISTINCT A.CEP_D as zipcode,
        ST_FlipCoordinates(ST_Transform(A.geometry, 'epsg:29193','epsg:4326')) as geometry,
        A.NAME as street_name,
        A.LENGTH as street_length
        FROM tb_zipcode as A      
)

SELECT * FROM zip_left 
UNION 
SELECT * FROM zip_right
"""

df_zipcode = spark.sql(Q_ZIPCODE_CONVERSION)

# dropping duplicates:
df_zipcode = df_zipcode.drop_duplicates(subset=["zipcode"])

In [213]:
# resulting dataframe becomes:
df_zipcode

zipcode,geometry,street_name,street_length
01012030,LINESTRING (-46.6...,CAFE,21.804877000000001
01233001,LINESTRING (-46.6...,TUPI,16.602098000000002
01248050,LINESTRING (-46.6...,PENAPOLIS,17.203655999999999
01303040,LINESTRING (-46.6...,GRAVATAI,141.574693000000000
01439020,LINESTRING (-46.6...,JAMAICA,94.675585999999996


In [214]:
# number of distinct zip codes:
df_zipcode.count()

44886

In [215]:
# registering to SQL context:
df_zipcode.createOrReplaceTempView("tb_zipcode")

## 1.6 Census Codebook
The codebook is a master file that contains the name of the variables in respect to their original locations and meaning in the raw data. We will use these to give more meaningful names to the many census variables.

In [217]:
# loading the codebook from the census data
DATA_DOC_DIR = "../references/documentation/"

df_codebook = spark.read.json(DATA_DOC_DIR + "ibge/codebook_features_selected.json")

In [218]:
# we will normalize the data in the codebook such that we can use it to match the variables in the other datasets:
new_cols = {
    "Descrição da Variável": "variable_description",
    "Nome da Variável": "variable_name",
    "Tabela": "dataset",
}

for original_name, new_name in new_cols.items():
    df_codebook = df_codebook.withColumnRenamed(original_name, new_name)

In [219]:
# codebook yields:
df_codebook.count()  # there are a total of 4107 variables in all the datasets

4107

In [220]:
# verifying the dataset:
df_codebook

dataset_name_pt,is_selected,normalized_variable,simplified_variable_name,variable_description_en,variable_description_pt,variable_name
Básico,1,sector_code,cod_setor,Sector code,Código do setor,Cod_setor
Básico,1,code_large_region...,cod_grandes_regioes,Code of Large Reg...,Código das Grande...,Cod_Grandes Regiões
Básico,1,name_large_region...,nome_grande_regiao,Name of large reg...,Nome das Grandes ...,Nome_Grande_Regiao
Básico,1,federation_unit_code,cod_uf,Federation Unit Code,Código da Unidade...,Cod_UF
Básico,1,name_federation_unit,nome_da_uf,Name of the Feder...,Nome da Unidade d...,Nome_da_UF


In [221]:
# building udfs from the utilities:
normalize_entities_udf = F.udf(normalize_entities)

df_codebook = df_codebook.withColumn(
    "dataset_normalized", normalize_entities_udf(F.col("dataset_name_pt"))
)

In [222]:
# fixing the order of columns for the codebook:
df_codebook_subset = df_codebook.select(
    "dataset_normalized", "simplified_variable_name", "normalized_variable"
)

# generating a lookup table base on the dataset : original_name relation
map_rdd = df_codebook_subset.rdd.groupBy(lambda row: row["dataset_normalized"]).map(
    lambda row: (row[0], {variable[1]: variable[2] for variable in row[1]})
)

codebook_lookup = dict(map_rdd.collect())

## 1.7 Census Data
The Census data itself is broken up in several different files that represent the different entities of the data. For example:

1. Domicilio{N}: refers to data at the `household`. Things like number of households in a sector, number of residents and such is located here;
2. Pessoa{N}: refers to the `person` data. Here we can find information about the sociodemographic make-up of the sector, such as gender distributions, number of children, race profiles, et cetera;

... and so on.

I will process all the files related to the city of São Paulo and gather them into the appropriate entities. I also went ahead and pre-selected several features from the codebook, such that we won't use all of the available features (`4107` according to the codebook).

# 2. Processing the Census Data

In [223]:
# defining a blacklist of column names to not convert data types:

blacklist = [
    "cod_bairro",
    "cod_distrito",
    "cod_grandes_regioes",
    "cod_meso",
    "cod_micro",
    "cod_municipio",
    "cod_rm",
    "cod_setor",
    "cod_subdistrito",
    "cod_uf",
    "nome_da_meso",
    "nome_da_micro",
    "nome_da_rm",
    "nome_da_uf",
    "nome_do_bairro",
    "nome_do_distrito",
    "nome_do_municipio",
    "nome_do_subdistrito",
    "nome_grande_regiao",
    "situacao_setor",
    "tipo_setor",
    "situacao",
    "setor_precoleta",
]

In [224]:
# list all the files available on the census data directory:
census_files = sorted(glob(RAW_DATA_DIR + "sp_census/universe_results/*.csv"))
BASE_PATH = RAW_DATA_DIR + "sp_census/universe_results/"

# saving the resulting dataframes to a processed stage for further handling:
PROCESSED_CENSUS_DATA_DIR = "../data/processed/sp_census/raw_datasets/"

# instantiating the UDF for replacing decimal separators:
replace_decimal_separator_udf = F.udf(replace_decimal_separator)
census_data = []

In [225]:
for file in tqdm(census_files):

    print(f"Processing file: {file}")

    # fixing names of the original tables
    dataset_name = (
        file.split("/")[-1].replace(".csv", "").replace("SP1", "").replace("_", "")
    )

    # normalizes the text data for the dataset
    dataset_name = normalize_entities(dataset_name)

    # loading the raw csv file
    df_temp = spark.read.csv(
        file,
        header=True,
        sep=";",
        inferSchema=False,
        encoding="ISO-8859-1",
    )

    # dropping duplicated columns (we can't do much about these)
    df_temp = drop_invalid_census_columns(df_temp, codebook_lookup, dataset_name)

    # retrieving the columns:
    original_columns = sorted(df_temp.columns)

    # normalize column names:
    normalized_columns = list(map(clean_census_column_name, original_columns))

    # looking up column names:
    new_columns = list(
        map(lambda col: codebook_lookup[dataset_name][col], normalized_columns)
    )

    # get columns with duplicates:
    cols_map = dict(zip(normalized_columns, new_columns))

    # iterating over the columns to replace with the new ones:
    for i in range(len(original_columns)):

        input_col = original_columns[i]
        check_col = clean_census_column_name(input_col)
        output_col = new_columns[i]

        try:

            if check_col not in blacklist:

                # renames the column:
                df_temp = df_temp.withColumnRenamed(input_col, output_col)

                # in this case, convert the data types to float after replacing decimal separators:
                df_temp = df_temp.withColumn(
                    output_col, replace_decimal_separator_udf(F.col(output_col))
                ).withColumn(output_col, F.col(output_col).cast("double"))

            else:
                df_temp = df_temp.withColumnRenamed(input_col, output_col)

        except Exception as e:
            print(f"Dropping column {col} as it could not be processed: {e}")

            df_temp.drop(input_col)

    # using the helper function to save the file:
    OUTPUT_NAME = f"tb_{dataset_name}"

    save_to_filesystem(
        df_temp, PROCESSED_CENSUS_DATA_DIR, OUTPUT_NAME, OUTPUT_NAME + ".parquet"
    )

  0%|          | 0/26 [00:00<?, ?it/s]

Processing file: ../data/raw/sp_census/universe_results/Basico_SP1.csv
Dropping columns [] - not present in the codebook
Dropping columns [] for being duplicates


  4%|▍         | 1/26 [00:00<00:23,  1.09it/s]

Processing file: ../data/raw/sp_census/universe_results/Domicilio01_SP1.csv
Dropping columns [] - not present in the codebook
Dropping columns ['V026', 'V120', 'V202', 'V206', 'V025', 'V198', 'V041', 'V203', 'V204', 'V119', 'V126', 'V125', 'V039', 'V208', 'V004', 'V139', 'V035'] for being duplicates


  8%|▊         | 2/26 [00:44<10:24, 26.03s/it]

...

 92%|█████████▏| 24/26 [08:40<00:25, 12.86s/it]

Processing file: ../data/raw/sp_census/universe_results/Responsavel02_SP1.csv
Dropping columns [] - not present in the codebook
Dropping columns [] for being duplicates


 96%|█████████▌| 25/26 [09:20<00:20, 20.80s/it]

Processing file: ../data/raw/sp_census/universe_results/ResponsavelRenda_SP1.csv
Dropping columns [] - not present in the codebook
Dropping columns ['V052', 'V085', 'V019', 'V118', 'V063', 'V129'] for being duplicates


100%|██████████| 26/26 [09:31<00:00, 21.97s/it]


# 3. Summarizing the Polygon Data
In order to simplify the operations with Polygon data at the lowest level (sector), we will add a helper column as an approximation of the geospatial reference of the sector by calculating its centroid. This will allow simpler joins later on at a reasonable level of approximation due to the comparatively small area of the census sectors.

We will also use these to generate the features themselves at different levels. 

In [226]:
# registering dataframes as temporary table views -- will allow us to access them in SQL Context:
df_sector.createOrReplaceTempView("tb_census_sector")

In [227]:
# adding the centroid and area of the polygons:
Q_ADD_GEOM = """
SELECT 
    A.*,
    ST_Centroid(A.geometry) as sector_centroid,
    ST_X(ST_Centroid(A.geometry)) as centroid_longitude,
    ST_Y(ST_Centroid(A.geometry)) as centroid_latitude
FROM tb_census_sector as A
"""

# adding the geometry:
df_sector = spark.sql(Q_ADD_GEOM)

In [228]:
# verifying the resulting schema:
df_sector.printSchema()  # looks good to go

root
 |-- id: string (nullable = true)
 |-- geometry: geometry (nullable = true)
 |-- sector_code: string (nullable = true)
 |-- city_code: string (nullable = true)
 |-- neighborhood_code: string (nullable = true)
 |-- city: string (nullable = true)
 |-- neighborhood: string (nullable = true)
 |-- sector_type: string (nullable = true)
 |-- sector_centroid: geometry (nullable = false)
 |-- centroid_longitude: double (nullable = true)
 |-- centroid_latitude: double (nullable = true)



# 4. Selecting the target features

In the codebook file provided, I had already selected several features of interest as a basis for this project. As it is originally intended for use in the Real State industry, the selection reflect that. Anyone interested in other kinds of features is free to create a selection of his or her own and use the following code as reference. The original files that hold the variables of interest are the following:

1. `Basico`
2. `Domicilio01`
3. `Domicilio02`
4. `Pessoa11`
5. `Pessoa12`
6. `Pessoa13`
7. `Responsavel02`

In [229]:
# specifying a directory for the census data:
CENSUS_DATA_DIR = "../data/processed/sp_census/raw_datasets/"

# basico - basic entity:
tb_basico = spark.read.parquet(CENSUS_DATA_DIR + "tb_basico.parquet")

# domicilio - housing entities:
tb_domicilio01 = spark.read.parquet(CENSUS_DATA_DIR + "tb_domicilio01.parquet")
tb_domicilio02 = spark.read.parquet(CENSUS_DATA_DIR + "tb_domicilio02.parquet")

# pessoa - person entities:
tb_pessoa01 = spark.read.parquet(CENSUS_DATA_DIR + "tb_pessoa01.parquet")
tb_pessoa11 = spark.read.parquet(CENSUS_DATA_DIR + "tb_pessoa11.parquet")
tb_pessoa12 = spark.read.parquet(CENSUS_DATA_DIR + "tb_pessoa12.parquet")
tb_pessoa13 = spark.read.parquet(CENSUS_DATA_DIR + "tb_pessoa13.parquet")

# resposavel - household responsible entities:
tb_responsible02 = spark.read.parquet(CENSUS_DATA_DIR + "tb_responsavel02.parquet")

In [230]:
# listing the number of census sectors:
tb_basico.count()  # joins show end up with the same number of rows (distinct sectors)

18363

In [231]:
# joining all the datasets before selecting:
tb_census = tb_basico.join(
    tb_domicilio01, on=["sector_code", "sector_situation_code"], how="left"
)
tb_census = tb_census.join(
    tb_domicilio02, on=["sector_code", "sector_situation_code"], how="left"
)
tb_census = tb_census.join(
    tb_pessoa11, on=["sector_code", "sector_situation_code"], how="left"
)
tb_census = tb_census.join(
    tb_pessoa12, on=["sector_code", "sector_situation_code"], how="left"
)
tb_census = tb_census.join(
    tb_responsible02, on=["sector_code", "sector_situation_code"], how="left"
)

tb_census = tb_census.join(
    tb_pessoa01, on=["sector_code", "sector_situation_code"], how="left"
)

In [233]:
# verifying the data integrity:
assert (
    tb_basico.count() == tb_census.count()
), "There are more rows than expected in the final file"

In [234]:
# selecting the columns that are relevant from the codebook:
col_selection = df_codebook.filter(F.col("is_selected") == 1)

In [235]:
# keeping the columns of interest:
cols_to_keep = get_column_values(col_selection, "normalized_variable")

In [236]:
# dropping the columns that are not part of the selection:
tb_census = tb_census.drop(
    *[col for col in tb_census.columns if col not in cols_to_keep]
)

# 5. Preparing features at the units of interest level

As previously described in the `1.0 Data Understanding - All Datasets.ipynb` notebook, we have several units of interest for the datasets we are working with. These are the following:

1. `sector`: the lowest unit of measurement for the Brazilian Census, which is one of most important geospatially referenced datasets we will be working with;
2. `zipcode`: zip codes in the city of São Paulo can be roughly approximated to an entire street (also called a logradouro);
3. `area_of_ponderation`: areas of ponderation are contiguous groups of census sectors;
4. `neighborhoods`: areas that are often (but not directly) related to the neighborhoods of the city;
5. `districts`: districts are administrative regions defined by law (and thus, won't change much over time), used to allocate resources by the City Hall;

The map below illustrates this relationship:

![Map illustrating the different layers](../references/img/layers_map_example.png)

We will aggregate the features at the levels above and maintain them in their raw state. Note: the zip code data will be processed in a different notebook as it requires a different processing approach.

In [238]:
# adding the geometry dataframes to the SQL Context:
df_ap.createOrReplaceTempView("tb_ponderation")
df_nb.createOrReplaceTempView("tb_neighborhood")
df_district.createOrReplaceTempView("tb_district")

In [239]:
# adding the raw features to the geometry:
df_sector_features = (
    df_sector.join(
        tb_census, (df_sector.sector_code == tb_census.sector_code), how="left"
    )
    .drop(tb_census.neighborhood_code)
    .drop(tb_census.sector_code)
)

In [240]:
# retrieving the number of rows after the join:
df_sector_features.count()

18953

In [241]:
# listing columns with null values:
get_null_columns(df_sector_features)

-RECORD 0----------------------------------------------------------------------------------------------------------------
 id                                                                                                                | 0   
 geometry                                                                                                          | 0   
 sector_code                                                                                                       | 0   
 city_code                                                                                                         | 0   
 neighborhood_code                                                                                                 | 0   
 city                                                                                                              | 0   
 neighborhood                                                                                                      | 0   
 sector_type            

In [242]:
# dropping null rows from the join (represent sectors that are not present in the area of study):
df_sector_features = df_sector_features.na.drop(
    subset=["code_large_regions_geographical_regions"]
)

In [243]:
# removing the missing values gets us back to the correct number (the ones that match the census exactly)
df_sector_features.count()

18363

In [244]:
# adding the resulting dataframe to the SQL Context:
df_sector_features.createOrReplaceTempView("tb_sector_features")

In [245]:
# adding the sectors back to the main dataframe:
df_sector_final = df_sector_features.join(
    df_sector.drop(
        "centroid_latitude",
        "centroid_longitude",
        "city_code",
        "city",
        "geometry",
        "id",
        "neighborhood_code",
        "neighborhood",
        "sector_centroid",
        "sector_type",
    ),
    on=["sector_code"],
    how="inner",
)

In [246]:
# save the results to the specified directory:
SECTOR_OUTPUT = f"sectors/tb_sectors_census"
PROCESSED_UNITS_OF_INTEREST = "../data/processed/sp_census/units_of_interest/"

save_to_filesystem(
    df_sector_final,
    PROCESSED_UNITS_OF_INTEREST,
    SECTOR_OUTPUT,
    SECTOR_OUTPUT + ".parquet",
)

True

In [247]:
SECTOR_NO_GEO = f"sectors/tb_sectors_census_no_geo"
PROCESSED_UNITS_OF_INTEREST = "../data/processed/sp_census/units_of_interest/"

df_sector_no_geo = df_sector.drop("geometry")

save_to_filesystem(
    df_sector_no_geo,
    PROCESSED_UNITS_OF_INTEREST,
    SECTOR_NO_GEO,
    SECTOR_NO_GEO + ".parquet",
)

True

## 5.1 Area of Ponderation

In [248]:
# adding the centroid and area of the polygons:
Q_AP_MATCH = """
SELECT 
    A.ponderation_area_code,
    B.sector_code
FROM tb_ponderation as A, tb_sector_features as B
WHERE ST_Contains(A.geometry, B.sector_centroid)
"""

# matching the areas of ponderation to their sectors:
df_ap_match = spark.sql(Q_AP_MATCH)

In [249]:
# adding the resulting dataframe to the SQL Context:
df_ap_match.createOrReplaceTempView("tb_ap_match")

In [250]:
# adding the features to the ponderation area back:
Q_AP_RAW_FEATURES = """
SELECT
    A.ponderation_area_code,
    A.sector_code,
    B.*
FROM tb_ap_match as A
LEFT JOIN tb_sector_features as B
ON A.sector_code = B.sector_code
"""

# reading back the dataset:
df_ap_raw_features = spark.sql(Q_AP_RAW_FEATURES)

# dropping the geometry columns and looking at the results:
df_ap_raw_features = df_ap_raw_features.drop("geometry", "sector_centroid")

df_ap_raw_features

ponderation_area_code,sector_code,id,sector_code,city_code,neighborhood_code,city,neighborhood,sector_type,centroid_longitude,centroid_latitude,sector_situation_code,code_large_regions_geographical_regions,name_large_regions_geographical_regions,federation_unit_code,name_federation_unit,mesoregion_code,name_mesoregion,microregion_code,name_microregion,metropolitan_region_code_or_ride,name_metropolitan_region_or_ride,code_municipality,name_municipality,district_code,district_name,subdistrict_code,name_sub_district,name_neighborhood,sector_type_code,permanent_private_households_or_persons_responsible_for_permanent_private_households,residents_permanent_private_households_or_population_residing_permanent_private_households,average_number_dwellers_permanent_private_households_obtained_by_var2_division_by_var1,value_average_monthly_nominal_income_persons_responsible_for_permanent_private_households_with_and_without_income,permanent_private_households_and_disposed,permanent_private_households_acquisition,permanent_private_households_rented,permanent_private_households_with_3_bathrooms_residents,permanent_private_households_without_exclusive_use_bathroom_residents,permanent_private_households_with_electricity,permanent_private_households_with_1_resident,permanent_private_households_with_2_residents,permanent_private_households_with_3_residents,permanent_private_households_with_4_residents,permanent_private_households_with_5_residents,permanent_private_households_with_6_residents,permanent_private_households_with_7_residents,permanent_private_households_with_8_residents,permanent_private_households_with_9_residents,permanent_private_households_with_10_or_more_residents,permanent_private_home_type_households_and_clear,permanent_private_home_type_households_and_acquisition,permanent_private_household_homes_rented,residents_private_households_and_collective_households,residents_permanent_household_private_households,residents_permanent_private_households_apartment_type,residents_permanent_permanent_households,dwellers_permanent_private_households_and_acquisition,residents_permanent_private_households_rented,children_only_from_person_responsible_private_households,household_employees_private_households_males,relatives_household_employees_private_households_male,household_employees_private_households,relatives_domestic_employees_as_private_households_female,literacy_responsible_persons,alphabetized_people_with_5_or_more_years_age
3550308005039,355030801000005,108943,355030801000005,3550308,,SÃO PAULO,,URBANO,-46.57065666912533,-23.571444624386825,1,3,Região Sudeste,35,São Paulo,3515,Metropolitana de ...,35061,São Paulo,20,RM São Paulo,3550308,SÃO PAULO,355030801,ÁGUA RASA,35503080100,ÁGUA RASA,SÃO PAULO (todos ...,0,240.0,754.0,3.14,1416.9,105.0,4.0,112.0,20.0,1.0,240.0,30.0,60.0,65.0,38.0,34.0,10.0,1.0,1.0,0.0,1.0,98.0,4.0,105.0,754.0,716.0,30.0,340.0,15.0,354.0,46.0,0.0,0.0,1.0,0.0,232.0,678.0
3550308005106,355030802000028,109096,355030802000028,3550308,,SÃO PAULO,,URBANO,-46.699390931740915,-23.54192792142038,1,3,Região Sudeste,35,São Paulo,3515,Metropolitana de ...,35061,São Paulo,20,RM São Paulo,3550308,SÃO PAULO,355030802,ALTO DE PINHEIROS,35503080200,ALTO DE PINHEIROS,SÃO PAULO (todos ...,0,177.0,394.0,2.23,5017.33,120.0,10.0,36.0,31.0,0.0,177.0,46.0,69.0,42.0,16.0,4.0,0.0,0.0,0.0,0.0,0.0,37.0,3.0,5.0,394.0,120.0,274.0,276.0,19.0,80.0,12.0,0.0,0.0,4.0,0.0,176.0,380.0
3550308005029,355030806000016,109560,355030806000016,3550308,,SÃO PAULO,,URBANO,-46.67432775353449,-23.52900256909393,1,3,Região Sudeste,35,São Paulo,3515,Metropolitana de ...,35061,São Paulo,20,RM São Paulo,3550308,SÃO PAULO,355030806,BARRA FUNDA,35503080600,BARRA FUNDA,SÃO PAULO (todos ...,0,135.0,356.0,2.64,3564.83,80.0,10.0,38.0,22.0,0.0,135.0,27.0,45.0,27.0,25.0,9.0,1.0,1.0,0.0,0.0,0.0,36.0,4.0,6.0,356.0,156.0,200.0,214.0,32.0,88.0,11.0,0.0,0.0,3.0,0.0,135.0,338.0
3550308005016,355030808000029,109753,355030808000029,3550308,,SÃO PAULO,,URBANO,-46.60066022961847,-23

In [251]:
# verifying the data integrity:
assert (
    df_ap_raw_features.count() == df_ap_match.count()
), "There are more rows than expected in the final file"

### 5.1.1 Aggregating Raw Area of Ponderation Features

In [253]:
# let's aggregate the data from the census sectors onto the area of ponderation level
df_ap_agg = df_ap_raw_features.groupby("ponderation_area_code").agg(
    F.sum(
        F.col(
            "permanent_private_households_or_persons_responsible_for_permanent_private_households"
        )
    ).alias("total_private_households"),
    F.sum(
        F.col(
            "residents_permanent_private_households_or_population_residing_permanent_private_households"
        )
    ).alias("total_population_private_households"),
    F.avg(
        F.col(
            "average_number_dwellers_permanent_private_households_obtained_by_var2_division_by_var1"
        )
    ).alias("average_number_residents_per_household"),
    F.avg(
        F.col(
            "value_average_monthly_nominal_income_persons_responsible_for_permanent_private_households_with_and_without_income"
        )
    ).alias("average_monthly_income"),
    F.sum(F.col("permanent_private_households_and_disposed")).alias(
        "total_fully_owned_properties"
    ),
    F.sum(F.col("permanent_private_households_acquisition")).alias(
        "total_properties_in_acquisition"
    ),
    F.sum(F.col("permanent_private_households_rented")).alias(
        "total_rented_properties"
    ),
    F.sum(F.col("permanent_private_households_with_3_bathrooms_residents")).alias(
        "total_properties_with_3_bathrooms"
    ),
    F.sum(
        F.col("permanent_private_households_without_exclusive_use_bathroom_residents")
    ).alias("total_properties_without_private_bathrooms"),
    F.sum(F.col("permanent_private_households_with_electricity")).alias(
        "total_households_with_electricity"
    ),
    F.sum(F.col("permanent_private_households_with_1_resident")).alias(
        "total_households_with_1_resident"
    ),
    F.sum(F.col("permanent_private_households_with_2_residents")).alias(
        "total_households_with_2_residents"
    ),
    F.sum(F.col("permanent_private_households_with_3_residents")).alias(
        "total_households_with_3_residents"
    ),
    F.sum(F.col("permanent_private_households_with_4_residents")).alias(
        "total_households_with_4_residents"
    ),
    F.sum(F.col("permanent_private_households_with_5_residents")).alias(
        "total_households_with_5_residents"
    ),
    F.sum(F.col("permanent_private_households_with_6_residents")).alias(
        "total_households_with_6_residents"
    ),
    F.sum(F.col("permanent_private_households_with_7_residents")).alias(
        "total_households_with_7_residents"
    ),
    F.sum(F.col("permanent_private_households_with_8_residents")).alias(
        "total_households_with_8_residents"
    ),
    F.sum(F.col("permanent_private_households_with_9_residents")).alias(
        "total_households_with_9_residents"
    ),
    F.sum(F.col("permanent_private_households_with_10_or_more_residents")).alias(
        "total_households_with_10_residents_or_more"
    ),
    F.sum(F.col("permanent_private_home_type_households_and_clear")).alias(
        "total_fully_owned_houses"
    ),
    F.sum(F.col("permanent_private_home_type_households_and_acquisition")).alias(
        "total_houses_in_acquisition"
    ),
    F.sum(F.col("permanent_private_household_homes_rented")).alias(
        "total_rented_houses"
    ),
    F.sum(F.col("residents_private_households_and_collective_households")).alias(
        "total_residents_in_area"
    ),
    F.sum(F.col("residents_permanent_household_private_households")).alias(
        "total_residents_in_permanent_households"
    ),
    F.sum(F.col("residents_permanent_private_households_apartment_type")).alias(
        "total_residents_in_private_apartments"
    ),
    F.sum(F.col("dwellers_permanent_private_households_and_acquisition")).alias(
        "total_residents_properties_in_acquisition"
    ),
    F.sum(F.col("residents_permanent_private_households_rented")).alias(
        "total_residents_rented_properties"
    ),
    F.sum(F.col("children_only_from_person_responsible_private_households")).alias(
        "total_number_children"
    ),
    F.sum(
        F.col("household_employees_private_households_males")
        + F.col("household_employees_private_households")
    ).alias("total_number_house_workers"),
    F.sum(
        F.col("relatives_domestic_employees_as_private_households_female")
        + F.col("relatives_household_employees_private_households_male")
    ).alias("total_number_house_workers_parents"),
    F.sum(F.col("literacy_responsible_persons")).alias(
        "total_literate_household_leaders"
    ),
    F.sum(F.col("alphabetized_people_with_5_or_more_years_age")).alias(
        "total_literate_people"
    ),
)

# fixing the column names:
new_columns = list(map(lambda col: f"ponderation_area_{col}", df_ap_agg.columns))

for i in range(len(df_ap_agg.columns)):
    df_ap_agg = df_ap_agg.withColumnRenamed(df_ap_agg.columns[i], new_columns[i])

# removing the extraneous columns:
df_ap_agg = df_ap_agg.withColumnRenamed(
    "ponderation_area_ponderation_area_code", "ponderation_area_code"
)

In [254]:
# verifying the results:
df_ap_agg

ponderation_area_code,ponderation_area_total_private_households,ponderation_area_total_population_private_households,ponderation_area_average_number_residents_per_household,ponderation_area_average_monthly_income,ponderation_area_total_fully_owned_properties,ponderation_area_total_properties_in_acquisition,ponderation_area_total_rented_properties,ponderation_area_total_properties_with_3_bathrooms,ponderation_area_total_properties_without_private_bathrooms,ponderation_area_total_households_with_electricity,ponderation_area_total_households_with_1_resident,ponderation_area_total_households_with_2_residents,ponderation_area_total_households_with_3_residents,ponderation_area_total_households_with_4_residents,ponderation_area_total_households_with_5_residents,ponderation_area_total_households_with_6_residents,ponderation_area_total_households_with_7_residents,ponderation_area_total_households_with_8_residents,ponderation_area_total_households_with_9_residents,ponderation_area_total_households_with_10_residents_or_more,ponderation_area_total_fully_owned_houses,ponderation_area_total_houses_in_acquisition,ponderation_area_total_rented_houses,ponderation_area_total_residents_in_area,ponderation_area_total_residents_in_permanent_households,ponderation_area_total_residents_in_private_apartments,ponderation_area_total_residents_properties_in_acquisition,ponderation_area_total_residents_rented_properties,ponderation_area_total_number_children,ponderation_area_total_number_house_workers,ponderation_area_total_number_house_workers_parents,ponderation_area_total_literate_household_leaders,ponderation_area_total_literate_people
3550308005048,15002.0,38235.0,2.5446874999999993,3969.7853124999997,9386.0,952.0,3988.0,2225.0,40.0,15000.0,3365.0,4888.0,3414.0,2319.0,705.0,202.0,63.0,27.0,11.0,8.0,3544.0,125.0,1323.0,38331.0,15543.0,21665.0,2400.0,9727.0,1490.0,143.0,6.0,14880.0,36173.0
3550308005045,8400.0,23519.0,2.801219512195122,3996.8287804878046,5266.0,938.0,1936.0,1359.0,104.0,8400.0,1303.0,2554.0,2197.0,1637.0,464.0,148.0,56.0,22.0,10.0,9.0,1982.0,97.0,1071.0,23546.0,10093.0,12905.0,2631.0,5466.0,1000.0,50.0,9.0,8346.0,22045.0
3550308005031,8835.0,26619.0,3.0826829268292677,2033.2802439024392,5135.0,622.0,2649.0,838.0,115.0,8830.0,1311.0,2339.0,2239.0,1743.0,739.0,253.0,100.0,54.0,23.0,34.0,3262.0,90.0,1929.0,26730.0,17762.0,7808.0,1814.0,8588.0,1327.0,30.0,1.0,8685.0,24533.0
3550308005158,8666.0,26741.0,3.0878787878787883,1294.025151515151,5434.0,410.0,2413.0,538.0,41.0,8664.0,1138.0,2198.0,2254.0,1800.0,787.0,294.0,109.0,40.0,24.0,22.0,5171.0,150.0,2298.0,26774.0,24925.0,1734.0,1277.0,7321.0,1324.0,17.0,0.0,8436.0,24581.0
3550308005102,9382.0,20499.0,2.219795918367347,5673.932040816328,5634.0,576.0,2625.0,2097.0,20.0,9382.0,3161.0,3151.0,1726.0,992.0,267.0,57.0,18.0,7.0,1.0,2.0,1020.0,16.0,276.0,20548.0,3842.0,16260.0,1211.0,5496.0,793.0,206.0,13.0,9365.0,19615.0


In [255]:
# adding the resulting dataframe to the SQL Context:
df_ap_agg.createOrReplaceTempView("tb_ponderation_features")

# generating the final level of aggregation for the census features:
Q_AP_GEOM = """
SELECT
    A.*,
    B.geometry,
    ST_Centroid(B.geometry) as ponderation_area_centroid
FROM tb_ponderation_features as A
LEFT JOIN tb_ponderation as B 
ON A.ponderation_area_code = B.ponderation_area_code
"""

df_ap_final = spark.sql(Q_AP_GEOM)

In [256]:
# looking at the results:
df_ap_final

ponderation_area_code,ponderation_area_total_private_households,ponderation_area_total_population_private_households,ponderation_area_average_number_residents_per_household,ponderation_area_average_monthly_income,ponderation_area_total_fully_owned_properties,ponderation_area_total_properties_in_acquisition,ponderation_area_total_rented_properties,ponderation_area_total_properties_with_3_bathrooms,ponderation_area_total_properties_without_private_bathrooms,ponderation_area_total_households_with_electricity,ponderation_area_total_households_with_1_resident,ponderation_area_total_households_with_2_residents,ponderation_area_total_households_with_3_residents,ponderation_area_total_households_with_4_residents,ponderation_area_total_households_with_5_residents,ponderation_area_total_households_with_6_residents,ponderation_area_total_households_with_7_residents,ponderation_area_total_households_with_8_residents,ponderation_area_total_households_with_9_residents,ponderation_area_total_households_with_10_residents_or_more,ponderation_area_total_fully_owned_houses,ponderation_area_total_houses_in_acquisition,ponderation_area_total_rented_houses,ponderation_area_total_residents_in_area,ponderation_area_total_residents_in_permanent_households,ponderation_area_total_residents_in_private_apartments,ponderation_area_total_residents_properties_in_acquisition,ponderation_area_total_residents_rented_properties,ponderation_area_total_number_children,ponderation_area_total_number_house_workers,ponderation_area_total_number_house_workers_parents,ponderation_area_total_literate_household_leaders,ponderation_area_total_literate_people,geometry,ponderation_area_centroid
3550308005045,8400.0,23519.0,2.801219512195122,3996.8287804878046,5266.0,938.0,1936.0,1359.0,104.0,8400.0,1303.0,2554.0,2197.0,1637.0,464.0,148.0,56.0,22.0,10.0,9.0,1982.0,97.0,1071.0,23546.0,10093.0,12905.0,2631.0,5466.0,1000.0,50.0,9.0,8346.0,22045.0,POLYGON ((-46.620...,POINT (-46.619835...
3550308005048,15002.0,38235.0,2.5446874999999993,3969.7853124999997,9386.0,952.0,3988.0,2225.0,40.0,15000.0,3365.0,4888.0,3414.0,2319.0,705.0,202.0,63.0,27.0,11.0,8.0,3544.0,125.0,1323.0,38331.0,15543.0,21665.0,2400.0,9727.0,1490.0,143.0,6.0,14880.0,36173.0,POLYGON ((-46.627...,POINT (-46.637962...
3550308005031,8835.0,26619.0,3.0826829268292677,2033.2802439024392,5135.0,622.0,2649.0,838.0,115.0,8830.0,1311.0,2339.0,2239.0,1743.0,739.0,253.0,100.0,54.0,23.0,34.0,3262.0,90.0,1929.0,26730.0,17762.0,7808.0,1814.0,8588.0,1327.0,30.0,1.0,8685.0,24533.0,POLYGON ((-46.596...,POINT (-46.607031...
3550308005102,9382.0,20499.0,2.219795918367347,5673.932040816328,5634.0,576.0,2625.0,2097.0,20.0,9382.0,3161.0,3151.0,1726.0,992.0,267.0,57.0,18.0,7.0,1.0,2.0,1020.0,16.0,276.0,20548.0,3842.0,16260.0,1211.0,5496.0,793.0,206.0,13.0,9365.0,19615.0,POLYGON ((-46.684...,POINT (-46.677750...
3550308005158,8666.0,26741.0,3.0878787878787883,1294.025151515151,5434.0,410.0,2413.0,538.0,41.0,8664.0,1138.0,2198.0,2254.0,1800.0,787.0,294.0,109.0,40.0,24.0,22.0,5171.0,150.0,2298.0,26774.0,24925.0,1734.0,1277.0,7321.0,1324.0,17.0,0.0,8436.0,24581.0,POLYGON ((-46.526...,POINT (-46.540679...


In [257]:
# looking at the schema:
df_ap_final.printSchema()

root
 |-- ponderation_area_code: string (nullable = true)
 |-- ponderation_area_total_private_households: double (nullable = true)
 |-- ponderation_area_total_population_private_households: double (nullable = true)
 |-- ponderation_area_average_number_residents_per_household: double (nullable = true)
 |-- ponderation_area_average_monthly_income: double (nullable = true)
 |-- ponderation_area_total_fully_owned_properties: double (nullable = true)
 |-- ponderation_area_total_properties_in_acquisition: double (nullable = true)
 |-- ponderation_area_total_rented_properties: double (nullable = true)
 |-- ponderation_area_total_properties_with_3_bathrooms: double (nullable = true)
 |-- ponderation_area_total_properties_without_private_bathrooms: double (nullable = true)
 |-- ponderation_area_total_households_with_electricity: double (nullable = true)
 |-- ponderation_area_total_households_with_1_resident: double (nullable = true)
 |-- ponderation_area_total_households_with_2_residents: doubl

### 5.1.2 Exporting Area of Ponderation level Results

In [258]:
# save the results to the specified directory:
AP_OUTPUT = f"areas_of_ponderation/tb_area_ponderation_census"
PROCESSED_UNITS_OF_INTEREST = "../data/processed/sp_census/units_of_interest/"

save_to_filesystem(
    df_ap_final, PROCESSED_UNITS_OF_INTEREST, AP_OUTPUT, AP_OUTPUT + ".parquet"
)

True

In [259]:
# saving without the geometry:
df_ap_no_geo = df_ap_final.drop("geometry", "ponderation_area_centroid")

AP_NO_GEO = f"areas_of_ponderation/tb_area_ponderation_census_no_geo"

save_to_filesystem(
    df_ap_final, PROCESSED_UNITS_OF_INTEREST, AP_NO_GEO, AP_NO_GEO + ".parquet"
)

True

## 5.2 Neighborhoods

In [260]:
# adding the centroid and area of the polygons:
Q_NEIGHBORHOOD_MATCH = """
SELECT 
    A.Name as neighborhood_name,
    B.sector_code
FROM tb_neighborhood as A, tb_sector_features as B
WHERE ST_Contains(A.geometry, B.sector_centroid)
"""

# matching the areas of ponderation to their sectors:
df_nb_match = spark.sql(Q_NEIGHBORHOOD_MATCH)

# adding the resulting dataframe to the SQL Context:
df_nb_match.createOrReplaceTempView("tb_nb_match")

In [261]:
# adding the features to the ponderation area back:
Q_NB_RAW_FEATURES = """
SELECT
    A.neighborhood_name,
    A.sector_code,
    B.*
FROM tb_nb_match as A
LEFT JOIN tb_sector_features as B
ON A.sector_code = B.sector_code
"""

# reading back the dataset:
df_nb_raw_features = spark.sql(Q_NB_RAW_FEATURES)

# dropping the geometry columns and looking at the results:
df_nb_raw_features = df_nb_raw_features.drop("geometry", "sector_centroid")

In [262]:
# verifying the data integrity:
assert (
    df_nb_raw_features.count() == df_nb_match.count()
), "There are more rows than expected in the final file"

### 5.2.1 Aggregating Raw Neighborhood Features

In [263]:
# let's aggregate the data from the census sectors onto the area of ponderation level
df_nb_agg = df_nb_raw_features.groupby("neighborhood_name").agg(
    F.sum(
        F.col(
            "permanent_private_households_or_persons_responsible_for_permanent_private_households"
        )
    ).alias("total_private_households"),
    F.sum(
        F.col(
            "residents_permanent_private_households_or_population_residing_permanent_private_households"
        )
    ).alias("total_population_private_households"),
    F.avg(
        F.col(
            "average_number_dwellers_permanent_private_households_obtained_by_var2_division_by_var1"
        )
    ).alias("average_number_residents_per_household"),
    F.avg(
        F.col(
            "value_average_monthly_nominal_income_persons_responsible_for_permanent_private_households_with_and_without_income"
        )
    ).alias("average_monthly_income"),
    F.sum(F.col("permanent_private_households_and_disposed")).alias(
        "total_fully_owned_properties"
    ),
    F.sum(F.col("permanent_private_households_acquisition")).alias(
        "total_properties_in_acquisition"
    ),
    F.sum(F.col("permanent_private_households_rented")).alias(
        "total_rented_properties"
    ),
    F.sum(F.col("permanent_private_households_with_3_bathrooms_residents")).alias(
        "total_properties_with_3_bathrooms"
    ),
    F.sum(
        F.col("permanent_private_households_without_exclusive_use_bathroom_residents")
    ).alias("total_properties_without_private_bathrooms"),
    F.sum(F.col("permanent_private_households_with_electricity")).alias(
        "total_households_with_electricity"
    ),
    F.sum(F.col("permanent_private_households_with_1_resident")).alias(
        "total_households_with_1_resident"
    ),
    F.sum(F.col("permanent_private_households_with_2_residents")).alias(
        "total_households_with_2_residents"
    ),
    F.sum(F.col("permanent_private_households_with_3_residents")).alias(
        "total_households_with_3_residents"
    ),
    F.sum(F.col("permanent_private_households_with_4_residents")).alias(
        "total_households_with_4_residents"
    ),
    F.sum(F.col("permanent_private_households_with_5_residents")).alias(
        "total_households_with_5_residents"
    ),
    F.sum(F.col("permanent_private_households_with_6_residents")).alias(
        "total_households_with_6_residents"
    ),
    F.sum(F.col("permanent_private_households_with_7_residents")).alias(
        "total_households_with_7_residents"
    ),
    F.sum(F.col("permanent_private_households_with_8_residents")).alias(
        "total_households_with_8_residents"
    ),
    F.sum(F.col("permanent_private_households_with_9_residents")).alias(
        "total_households_with_9_residents"
    ),
    F.sum(F.col("permanent_private_households_with_10_or_more_residents")).alias(
        "total_households_with_10_residents_or_more"
    ),
    F.sum(F.col("permanent_private_home_type_households_and_clear")).alias(
        "total_fully_owned_houses"
    ),
    F.sum(F.col("permanent_private_home_type_households_and_acquisition")).alias(
        "total_houses_in_acquisition"
    ),
    F.sum(F.col("permanent_private_household_homes_rented")).alias(
        "total_rented_houses"
    ),
    F.sum(F.col("residents_private_households_and_collective_households")).alias(
        "total_residents_in_area"
    ),
    F.sum(F.col("residents_permanent_household_private_households")).alias(
        "total_residents_in_permanent_households"
    ),
    F.sum(F.col("residents_permanent_private_households_apartment_type")).alias(
        "total_residents_in_private_apartments"
    ),
    F.sum(F.col("dwellers_permanent_private_households_and_acquisition")).alias(
        "total_residents_properties_in_acquisition"
    ),
    F.sum(F.col("residents_permanent_private_households_rented")).alias(
        "total_residents_rented_properties"
    ),
    F.sum(F.col("children_only_from_person_responsible_private_households")).alias(
        "total_number_children"
    ),
    F.sum(
        F.col("household_employees_private_households_males")
        + F.col("household_employees_private_households")
    ).alias("total_number_house_workers"),
    F.sum(
        F.col("relatives_domestic_employees_as_private_households_female")
        + F.col("relatives_household_employees_private_households_male")
    ).alias("total_number_house_workers_parents"),
    F.sum(F.col("literacy_responsible_persons")).alias(
        "total_literate_household_leaders"
    ),
    F.sum(F.col("alphabetized_people_with_5_or_more_years_age")).alias(
        "total_literate_people"
    ),
)

# fixing the column names:
new_columns = list(map(lambda col: f"neighborhood_{col}", df_nb_agg.columns))

for i in range(len(df_ap_agg.columns)):
    df_nb_agg = df_nb_agg.withColumnRenamed(df_nb_agg.columns[i], new_columns[i])

df_nb_agg = df_nb_agg.withColumnRenamed(
    "neighborhood_neighborhood_name", "neighborhood_name"
)

In [264]:
# adding the resulting dataframe to the SQL Context:
df_nb_agg.createOrReplaceTempView("tb_neighborhood_features")

# generating the final level of aggregation for the census features:
Q_NB_GEOM = """
SELECT
    A.*,
    B.geometry,
    ST_Centroid(B.geometry) as neighborhood_centroid
FROM tb_neighborhood_features as A
LEFT JOIN tb_neighborhood as B 
ON A.neighborhood_name = B.Name
"""

df_nb_final = spark.sql(Q_NB_GEOM)

### 5.2.2 Exporting Neighborhood level Results

In [265]:
# save the results to the specified directory:
NB_OUTPUT = f"neighborhoods/tb_neighborhood_census"
PROCESSED_UNITS_OF_INTEREST = "../data/processed/sp_census/units_of_interest/"

save_to_filesystem(
    df_nb_final, PROCESSED_UNITS_OF_INTEREST, NB_OUTPUT, NB_OUTPUT + ".parquet"
)

True

In [266]:
# saving the results the results without geometries:
NB_NO_GEO = f"neighborhoods/tb_neighborhood_census_no_geo"

df_nb_no_geo = df_nb_final.drop("geometry", "neighborhood_centroid")

save_to_filesystem(
    df_nb_no_geo, PROCESSED_UNITS_OF_INTEREST, NB_NO_GEO, NB_NO_GEO + ".parquet"
)

True

## 5.3 Districts

In [267]:
# adding the centroid and area of the polygons:
Q_DS_MATCH = """
SELECT 
    A.district_name,
    B.sector_code
FROM tb_district as A, tb_sector_features as B
WHERE ST_Contains(A.geometry, B.sector_centroid)
"""

# matching the areas of ponderation to their sectors:
df_ds_match = spark.sql(Q_DS_MATCH)

In [268]:
# adding the resulting dataframe to the SQL Context:
df_ds_match.createOrReplaceTempView("tb_ds_match")
df_sector_features = df_sector_features.drop("district_name")
df_sector_features.createOrReplaceTempView("tb_sector_features")

In [269]:
# adding the features to the ponderation area back:
Q_DS_RAW_FEATURES = """
SELECT
    A.district_name,
    A.sector_code,
    B.*
FROM tb_ds_match as A
LEFT JOIN tb_sector_features as B
ON A.sector_code = B.sector_code
"""

# reading back the dataset:
df_ds_raw_features = spark.sql(Q_DS_RAW_FEATURES)

# dropping the geometry columns and looking at the results:
df_ds_raw_features = df_ds_raw_features.drop("geometry", "sector_centroid")

### 5.3.1 Aggregating Raw District Features

In [270]:
# let's aggregate the data from the census sectors onto the area of ponderation level
df_ds_agg = df_ds_raw_features.groupby("district_name").agg(
    F.sum(
        F.col(
            "permanent_private_households_or_persons_responsible_for_permanent_private_households"
        )
    ).alias("total_private_households"),
    F.sum(
        F.col(
            "residents_permanent_private_households_or_population_residing_permanent_private_households"
        )
    ).alias("total_population_private_households"),
    F.avg(
        F.col(
            "average_number_dwellers_permanent_private_households_obtained_by_var2_division_by_var1"
        )
    ).alias("average_number_residents_per_household"),
    F.avg(
        F.col(
            "value_average_monthly_nominal_income_persons_responsible_for_permanent_private_households_with_and_without_income"
        )
    ).alias("average_monthly_income"),
    F.sum(F.col("permanent_private_households_and_disposed")).alias(
        "total_fully_owned_properties"
    ),
    F.sum(F.col("permanent_private_households_acquisition")).alias(
        "total_properties_in_acquisition"
    ),
    F.sum(F.col("permanent_private_households_rented")).alias(
        "total_rented_properties"
    ),
    F.sum(F.col("permanent_private_households_with_3_bathrooms_residents")).alias(
        "total_properties_with_3_bathrooms"
    ),
    F.sum(
        F.col("permanent_private_households_without_exclusive_use_bathroom_residents")
    ).alias("total_properties_without_private_bathrooms"),
    F.sum(F.col("permanent_private_households_with_electricity")).alias(
        "total_households_with_electricity"
    ),
    F.sum(F.col("permanent_private_households_with_1_resident")).alias(
        "total_households_with_1_resident"
    ),
    F.sum(F.col("permanent_private_households_with_2_residents")).alias(
        "total_households_with_2_residents"
    ),
    F.sum(F.col("permanent_private_households_with_3_residents")).alias(
        "total_households_with_3_residents"
    ),
    F.sum(F.col("permanent_private_households_with_4_residents")).alias(
        "total_households_with_4_residents"
    ),
    F.sum(F.col("permanent_private_households_with_5_residents")).alias(
        "total_households_with_5_residents"
    ),
    F.sum(F.col("permanent_private_households_with_6_residents")).alias(
        "total_households_with_6_residents"
    ),
    F.sum(F.col("permanent_private_households_with_7_residents")).alias(
        "total_households_with_7_residents"
    ),
    F.sum(F.col("permanent_private_households_with_8_residents")).alias(
        "total_households_with_8_residents"
    ),
    F.sum(F.col("permanent_private_households_with_9_residents")).alias(
        "total_households_with_9_residents"
    ),
    F.sum(F.col("permanent_private_households_with_10_or_more_residents")).alias(
        "total_households_with_10_residents_or_more"
    ),
    F.sum(F.col("permanent_private_home_type_households_and_clear")).alias(
        "total_fully_owned_houses"
    ),
    F.sum(F.col("permanent_private_home_type_households_and_acquisition")).alias(
        "total_houses_in_acquisition"
    ),
    F.sum(F.col("permanent_private_household_homes_rented")).alias(
        "total_rented_houses"
    ),
    F.sum(F.col("residents_private_households_and_collective_households")).alias(
        "total_residents_in_area"
    ),
    F.sum(F.col("residents_permanent_household_private_households")).alias(
        "total_residents_in_permanent_households"
    ),
    F.sum(F.col("residents_permanent_private_households_apartment_type")).alias(
        "total_residents_in_private_apartments"
    ),
    F.sum(F.col("dwellers_permanent_private_households_and_acquisition")).alias(
        "total_residents_properties_in_acquisition"
    ),
    F.sum(F.col("residents_permanent_private_households_rented")).alias(
        "total_residents_rented_properties"
    ),
    F.sum(F.col("children_only_from_person_responsible_private_households")).alias(
        "total_number_children"
    ),
    F.sum(
        F.col("household_employees_private_households_males")
        + F.col("household_employees_private_households")
    ).alias("total_number_house_workers"),
    F.sum(
        F.col("relatives_domestic_employees_as_private_households_female")
        + F.col("relatives_household_employees_private_households_male")
    ).alias("total_number_house_workers_parents"),
    F.sum(F.col("literacy_responsible_persons")).alias(
        "total_literate_household_leaders"
    ),
    F.sum(F.col("alphabetized_people_with_5_or_more_years_age")).alias(
        "total_literate_people"
    ),
)

# fixing the column names:
new_columns = list(map(lambda col: f"district_{col}", df_ds_agg.columns))

for i in range(len(df_ds_agg.columns)):
    df_ds_agg = df_ds_agg.withColumnRenamed(df_ds_agg.columns[i], new_columns[i])

df_ds_agg = df_ds_agg.withColumnRenamed("district_district_name", "district_name")

In [271]:
# adding the resulting dataframe to the SQL Context:
df_ds_agg.createOrReplaceTempView("tb_district_features")

# generating the final level of aggregation for the census features:
Q_DS_GEOM = """
SELECT
    A.*,
    B.geometry,
    ST_Centroid(B.geometry) as district_centroid
FROM tb_district_features as A
LEFT JOIN tb_district as B 
ON A.district_name = B.district_name
"""

df_ds_final = spark.sql(Q_DS_GEOM)

In [272]:
# verifying the results:
df_ds_final

district_name,district_total_private_households,district_total_population_private_households,district_average_number_residents_per_household,district_average_monthly_income,district_total_fully_owned_properties,district_total_properties_in_acquisition,district_total_rented_properties,district_total_properties_with_3_bathrooms,district_total_properties_without_private_bathrooms,district_total_households_with_electricity,district_total_households_with_1_resident,district_total_households_with_2_residents,district_total_households_with_3_residents,district_total_households_with_4_residents,district_total_households_with_5_residents,district_total_households_with_6_residents,district_total_households_with_7_residents,district_total_households_with_8_residents,district_total_households_with_9_residents,district_total_households_with_10_residents_or_more,district_total_fully_owned_houses,district_total_houses_in_acquisition,district_total_rented_houses,district_total_residents_in_area,district_total_residents_in_permanent_households,district_total_residents_in_private_apartments,district_total_residents_properties_in_acquisition,district_total_residents_rented_properties,district_total_number_children,district_total_number_house_workers,district_total_number_house_workers_parents,district_total_literate_household_leaders,district_total_literate_people,geometry,district_centroid
SAO RAFAEL,40502.0,143559.0,3.532713567839195,847.8482914572866,27110.0,2464.0,6924.0,1549.0,763.0,40462.0,3388.0,7565.0,10381.0,9718.0,5325.0,2248.0,1005.0,456.0,207.0,209.0,26233.0,972.0,6678.0,143656.0,133780.0,7195.0,9065.0,22839.0,8416.0,13.0,1.0,37766.0,122441.0,POLYGON ((-46.481...,POINT (-46.453991...
MARSILAC,2349.0,8185.0,3.5406666666666675,665.7364444444444,1594.0,59.0,138.0,69.0,12.0,2316.0,296.0,479.0,558.0,420.0,289.0,131.0,72.0,42.0,16.0,24.0,1586.0,24.0,136.0,8258.0,7949.0,134.0,194.0,453.0,572.0,2.0,0.0,2052.0,6794.0,POLYGON ((-46.609...,POINT (-46.707905...
RIO PEQUENO,36945.0,116877.0,3.1489534883720935,2701.8732558139513,24426.0,2887.0,7765.0,3716.0,1037.0,36906.0,4704.0,8978.0,9546.0,7779.0,3382.0,1367.0,584.0,295.0,144.0,135.0,17305.0,823.0,5904.0,117175.0,85463.0,25448.0,8373.0,23144.0,6687.0,199.0,16.0,35746.0,105454.0,POLYGON ((-46.744...,POINT (-46.758913...
CACHOEIRINHA,42898.0,144280.0,3.421146788990824,1233.538211009174,25236.0,3536.0,10259.0,1953.0,1357.0,42883.0,4540.0,9271.0,11184.0,9403.0,4686.0,2004.0,912.0,444.0,189.0,259.0,20875.0,431.0,8591.0,144378.0,113001.0,22846.0,11017.0,32889.0,9432.0,45.0,1.0,40781.0,126205.0,POLYGON ((-46.655...,POINT (-46.662130...
ERMELINO MATARAZZO,33961.0,113289.0,3.346988636363635,1104.565170454545,21736.0,1648.0,8434.0,1613.0,198.0,33950.0,3508.0,7124.0,8897.0,7970.0,3833.0,1516.0,625.0,267.0,128.0,93.0,20362.0,465.0,7916.0,113379.0,103586.0,7041.0,5157.0,26682.0,7218.0,59.0,10.0,32531.0,100122.0,POLYGON ((-46.469...,POINT (-46.484567...


### 5.3.2 Exporting District level Results

In [273]:
# save the results to the specified directory:
DS_OUTPUT = f"districts/tb_district_census"
PROCESSED_UNITS_OF_INTEREST = "../data/processed/sp_census/units_of_interest/"

save_to_filesystem(
    df_ds_final, PROCESSED_UNITS_OF_INTEREST, DS_OUTPUT, DS_OUTPUT + ".parquet"
)

True

In [274]:
# saving the results the results without geometries:
DS_NO_GEO = f"districts/tb_district_census_no_geo"

df_ds_no_geo = df_ds_final.drop("geometry", "district_centroid")

save_to_filesystem(
    df_ds_no_geo, PROCESSED_UNITS_OF_INTEREST, DS_NO_GEO, DS_NO_GEO + ".parquet"
)

True

## 5.4 Zip Code

The deal with the Zip codes is that, different from a polygon, it does not represent an enclosed structure. With that, with that, the approach we were developing before that included matching the centroids of the census sectors to the polygon in which they are located. The base shapes are `linestrings` instead. `linestrings` can intersect census sectors, for example. With that, we will have an `intersection` join instead of `containment` spatial join.

Why can't we just calculate the middle point from the line representing the sector instead and use it as a reference? That could certainly work, but it does not capture much of the variation in longer streets (and thus longer zip codes). Performing the crossing join is a way to capture more of the variation along the same street.

> **Note**: this can be quite an expensive operation, since we end up with a many-to-many relationship established. One street can cross multiple sectors and sectors can be crossed by multiple streets (zip codes). It should be taken into consideration. 

In [275]:
# calculating the
Q_ZIPCODE_MATCH = """
SELECT 
    A.zipcode,
    B.sector_code
FROM tb_zipcode as A, tb_sector_features as B
WHERE ST_Intersects(B.geometry, A.geometry)
"""

# matching the areas of ponderation to their sectors:
df_zipcode_match = spark.sql(Q_ZIPCODE_MATCH)

# adding the resulting dataframe to the SQL Context:
df_zipcode_match.createOrReplaceTempView("tb_zipcode_match")

In [276]:
# verifying the results:
df_zipcode_match

zipcode,sector_code
04886050,355030855000057
04891110,355030855000042
04888040,355030855000025
04891160,355030855000041
04888080,355030855000056


In [277]:
# counting the number of zipcodes
df_zipcode_match.count()

69557

In [278]:
# verifying the number of distinct zipcodes:
df_zipcode_match.select(
    F.countDistinct(F.col("zipcode")).alias("n_distinct_zipcodes")
)  # we lost some zip codes (about 114), that is good enough

n_distinct_zipcodes
44772


In [279]:
# adding the features to the ponderation area back:
Q_ZIPCODE_FEATURES = """
SELECT
    A.zipcode,
    A.sector_code,
    B.*
FROM tb_zipcode_match as A
LEFT JOIN tb_sector_features as B
ON A.sector_code = B.sector_code
"""

# reading back the dataset:
df_zipcode_raw_features = spark.sql(Q_ZIPCODE_FEATURES)

# dropping the geometry columns and looking at the results:
df_zipcode_raw_features = df_zipcode_raw_features.drop("geometry", "sector_centroid")

### 5.4.1 Aggregating the Zip Code Features

In [280]:
# let's aggregate the data from the census sectors onto the area of ponderation level
df_zipcode_agg = df_zipcode_raw_features.groupby("zipcode").agg(
    F.sum(
        F.col(
            "permanent_private_households_or_persons_responsible_for_permanent_private_households"
        )
    ).alias("total_private_households"),
    F.sum(
        F.col(
            "residents_permanent_private_households_or_population_residing_permanent_private_households"
        )
    ).alias("total_population_private_households"),
    F.avg(
        F.col(
            "average_number_dwellers_permanent_private_households_obtained_by_var2_division_by_var1"
        )
    ).alias("average_number_residents_per_household"),
    F.avg(
        F.col(
            "value_average_monthly_nominal_income_persons_responsible_for_permanent_private_households_with_and_without_income"
        )
    ).alias("average_monthly_income"),
    F.sum(F.col("permanent_private_households_and_disposed")).alias(
        "total_fully_owned_properties"
    ),
    F.sum(F.col("permanent_private_households_acquisition")).alias(
        "total_properties_in_acquisition"
    ),
    F.sum(F.col("permanent_private_households_rented")).alias(
        "total_rented_properties"
    ),
    F.sum(F.col("permanent_private_households_with_3_bathrooms_residents")).alias(
        "total_properties_with_3_bathrooms"
    ),
    F.sum(
        F.col("permanent_private_households_without_exclusive_use_bathroom_residents")
    ).alias("total_properties_without_private_bathrooms"),
    F.sum(F.col("permanent_private_households_with_electricity")).alias(
        "total_households_with_electricity"
    ),
    F.sum(F.col("permanent_private_households_with_1_resident")).alias(
        "total_households_with_1_resident"
    ),
    F.sum(F.col("permanent_private_households_with_2_residents")).alias(
        "total_households_with_2_residents"
    ),
    F.sum(F.col("permanent_private_households_with_3_residents")).alias(
        "total_households_with_3_residents"
    ),
    F.sum(F.col("permanent_private_households_with_4_residents")).alias(
        "total_households_with_4_residents"
    ),
    F.sum(F.col("permanent_private_households_with_5_residents")).alias(
        "total_households_with_5_residents"
    ),
    F.sum(F.col("permanent_private_households_with_6_residents")).alias(
        "total_households_with_6_residents"
    ),
    F.sum(F.col("permanent_private_households_with_7_residents")).alias(
        "total_households_with_7_residents"
    ),
    F.sum(F.col("permanent_private_households_with_8_residents")).alias(
        "total_households_with_8_residents"
    ),
    F.sum(F.col("permanent_private_households_with_9_residents")).alias(
        "total_households_with_9_residents"
    ),
    F.sum(F.col("permanent_private_households_with_10_or_more_residents")).alias(
        "total_households_with_10_residents_or_more"
    ),
    F.sum(F.col("permanent_private_home_type_households_and_clear")).alias(
        "total_fully_owned_houses"
    ),
    F.sum(F.col("permanent_private_home_type_households_and_acquisition")).alias(
        "total_houses_in_acquisition"
    ),
    F.sum(F.col("permanent_private_household_homes_rented")).alias(
        "total_rented_houses"
    ),
    F.sum(F.col("residents_private_households_and_collective_households")).alias(
        "total_residents_in_area"
    ),
    F.sum(F.col("residents_permanent_household_private_households")).alias(
        "total_residents_in_permanent_households"
    ),
    F.sum(F.col("residents_permanent_private_households_apartment_type")).alias(
        "total_residents_in_private_apartments"
    ),
    F.sum(F.col("dwellers_permanent_private_households_and_acquisition")).alias(
        "total_residents_properties_in_acquisition"
    ),
    F.sum(F.col("residents_permanent_private_households_rented")).alias(
        "total_residents_rented_properties"
    ),
    F.sum(F.col("children_only_from_person_responsible_private_households")).alias(
        "total_number_children"
    ),
    F.sum(
        F.col("household_employees_private_households_males")
        + F.col("household_employees_private_households")
    ).alias("total_number_house_workers"),
    F.sum(
        F.col("relatives_domestic_employees_as_private_households_female")
        + F.col("relatives_household_employees_private_households_male")
    ).alias("total_number_house_workers_parents"),
    F.sum(F.col("literacy_responsible_persons")).alias(
        "total_literate_household_leaders"
    ),
    F.sum(F.col("alphabetized_people_with_5_or_more_years_age")).alias(
        "total_literate_people"
    ),
)

# fixing the column names:
new_columns = list(map(lambda col: f"zipcode_{col}", df_zipcode_agg.columns))

for i in range(len(df_zipcode_agg.columns)):
    df_zipcode_agg = df_zipcode_agg.withColumnRenamed(
        df_zipcode_agg.columns[i], new_columns[i]
    )

df_zipcode_agg = df_zipcode_agg.withColumnRenamed("zipcode_zipcode", "zipcode")

In [281]:
# adding the resulting dataframe to the SQL Context:
df_zipcode_agg.createOrReplaceTempView("tb_zipcode_features")

# generating the final level of aggregation for the census features:
Q_ZIPCODE_GEOM = """
SELECT
    A.*,
    B.geometry,
    ST_Centroid(B.geometry) as district_centroid
FROM tb_zipcode_features as A
LEFT JOIN tb_zipcode as B 
ON A.zipcode = B.zipcode
"""

df_zipcode_final = spark.sql(Q_ZIPCODE_GEOM)

In [282]:
# verifying the results:
df_zipcode_final

zipcode,zipcode_total_private_households,zipcode_total_population_private_households,zipcode_average_number_residents_per_household,zipcode_average_monthly_income,zipcode_total_fully_owned_properties,zipcode_total_properties_in_acquisition,zipcode_total_rented_properties,zipcode_total_properties_with_3_bathrooms,zipcode_total_properties_without_private_bathrooms,zipcode_total_households_with_electricity,zipcode_total_households_with_1_resident,zipcode_total_households_with_2_residents,zipcode_total_households_with_3_residents,zipcode_total_households_with_4_residents,zipcode_total_households_with_5_residents,zipcode_total_households_with_6_residents,zipcode_total_households_with_7_residents,zipcode_total_households_with_8_residents,zipcode_total_households_with_9_residents,zipcode_total_households_with_10_residents_or_more,zipcode_total_fully_owned_houses,zipcode_total_houses_in_acquisition,zipcode_total_rented_houses,zipcode_total_residents_in_area,zipcode_total_residents_in_permanent_households,zipcode_total_residents_in_private_apartments,zipcode_total_residents_properties_in_acquisition,zipcode_total_residents_rented_properties,zipcode_total_number_children,zipcode_total_number_house_workers,zipcode_total_number_house_workers_parents,zipcode_total_literate_household_leaders,zipcode_total_literate_people,geometry,district_centroid
01012030,60.0,134.0,2.23,1287.17,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,134.0,null,null,null,null,null,null,null,null,121.0,LINESTRING (-46.6...,POINT (-46.635104...
01233001,559.0,1150.0,2.0549999999999997,3243.115,313.0,43.0,178.0,21.0,0.0,559.0,214.0,175.0,111.0,45.0,12.0,1.0,1.0,0.0,0.0,0.0,10.0,0.0,6.0,1166.0,37.0,1109.0,97.0,386.0,36.0,5.0,0.0,560.0,1118.0,LINESTRING (-46.6...,POINT (-46.659862...
01248050,262.0,895.0,3.42,13517.15,227.0,1.0,25.0,56.0,0.0,262.0,26.0,67.0,52.0,57.0,31.0,18.0,6.0,3.0,1.0,1.0,219.0,1.0,14.0,903.0,847.0,48.0,3.0,77.0,28.0,111.0,11.0,263.0,857.0,LINESTRING (-46.6...,POINT (-46.667042...
01303040,417.0,781.0,1.875,3028.6949999999997,182.0,60.0,146.0,44.0,0.0,417.0,210.0,135.0,39.0,16.0,10.0,3.0,2.0,0.0,0.0,2.0,8.0,0.0,3.0,801.0,61.0,717.0,91.0,280.0,21.0,5.0,0.0,416.0,763.0,LINESTRING (-46.6...,POINT (-46.647820...
01439020,421.0,986.0,2.4233333333333333,10323.269999999999,338.0,5.0,57.0,195.0,0.0,421.0,134.0,134.0,77.0,45.0,20.0,6.0,3.0,2.0,0.0,0.0,99.0,0.0,16.0,986.0,398.0,588.0,9.0,135.0,43.0,55.0,0.0,418.0,938.0,LINESTRING (-46.6...,POINT (-46.672204...


In [283]:
# save the results to the specified directory:
ZIP_OUTPUT = f"zipcodes/tb_zipcode_census"
PROCESSED_UNITS_OF_INTEREST = "../data/processed/sp_census/units_of_interest/"

save_to_filesystem(
    df_zipcode_final, PROCESSED_UNITS_OF_INTEREST, ZIP_OUTPUT, ZIP_OUTPUT + ".parquet"
)

True

In [ ]:
# save the results to the specified directory:
ZIP_NO_GEO = f"zipcodes/tb_zipcode_census_no_geo"
PROCESSED_UNITS_OF_INTEREST = "../data/processed/sp_census/units_of_interest/"

df_zip_no_geo = df_zipcode_final.drop("geometry")

save_to_filesystem(
    df_zip_no_geo, PROCESSED_UNITS_OF_INTEREST, ZIP_NO_GEO, ZIP_NO_GEO + ".parquet"
)